In [61]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [63]:
from src.inference import get_feature_store

In [64]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*28)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-04 02:23:21,140 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 02:23:21,141 INFO: Initializing external client
2025-03-04 02:23:21,141 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 02:23:21,862 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214668
Fetching data from 2025-02-04 07:23:21.140148+00:00 to 2025-03-04 06:23:21.140148+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.27s) 


In [65]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-02-20 21:00:00+00:00,210,0
1,2025-02-13 01:00:00+00:00,147,0
2,2025-03-04 04:00:00+00:00,209,0
3,2025-02-17 12:00:00+00:00,65,3
4,2025-02-23 20:00:00+00:00,56,0
...,...,...,...
169169,2025-02-26 08:00:00+00:00,243,1
169170,2025-02-26 00:00:00+00:00,187,0
169171,2025-02-07 16:00:00+00:00,257,0
169172,2025-02-21 17:00:00+00:00,254,2


In [66]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-04 08:00:00+00:00,2,0
1,2025-02-04 09:00:00+00:00,2,0
2,2025-02-04 10:00:00+00:00,2,0
3,2025-02-04 11:00:00+00:00,2,0
4,2025-02-04 12:00:00+00:00,2,0
...,...,...,...
168416,2025-03-04 02:00:00+00:00,263,6
168417,2025-03-04 03:00:00+00:00,263,3
168418,2025-03-04 04:00:00+00:00,263,8
168419,2025-03-04 05:00:00+00:00,263,18


In [67]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168421 entries, 0 to 169173
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         168421 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  168421 non-null  int32                  
 2   rides               168421 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 3.9 MB


In [68]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [69]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168421 entries, 0 to 169173
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         168421 non-null  datetime64[us]
 1   pickup_location_id  168421 non-null  int32         
 2   rides               168421 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 3.9 MB


In [70]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-02-20 21:00:00,210,0
1,2025-02-13 01:00:00,147,0
2,2025-03-04 04:00:00,209,0
3,2025-02-17 12:00:00,65,3
4,2025-02-23 20:00:00,56,0
...,...,...,...
169169,2025-02-26 08:00:00,243,1
169170,2025-02-26 00:00:00,187,0
169171,2025-02-07 16:00:00,257,0
169172,2025-02-21 17:00:00,254,2


In [71]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*27, step_size=23)

In [72]:
features

,rides_t-648,rides_t-647,rides_t-646,rides_t-645,rides_t-644,rides_t-643,rides_t-642,rides_t-641,rides_t-640,rides_t-639,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,210,2025-03-02 07:00:00
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,147,2025-02-28 05:00:00
2,0,9,0,0,3,7,7,1,5,0,...,2,14,11,14,26,6,23,1,209,2025-02-09 12:00:00
3,3,2,2,5,4,0,5,0,2,2,...,0,7,3,1,0,5,0,5,65,2025-03-01 18:00:00
4,0,0,0,0,0,0,0,0,1,0,...,0,0,1,1,0,0,1,0,56,2025-02-26 14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,2,5,1,0,14,22,23,2,6,1,...,24,1,9,18,1,13,9,34,74,2025-02-07 07:00:00
247,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,67,2025-02-25 17:00:00
248,3,1,0,2,0,0,0,2,1,0,...,0,0,1,2,2,0,1,1,62,2025-02-13 09:00:00
249,2,3,0,2,1,0,0,4,0,2,...,1,0,1,7,0,0,0,0,35,2025-02-19 12:00:00


In [90]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-04 02:28:40,840 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 02:28:40,844 INFO: Initializing external client
2025-03-04 02:28:40,845 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 02:28:41,665 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214668
Fetching data from 2025-02-03 07:28:40.839943+00:00 to 2025-03-04 06:28:40.839943+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.80s) 


In [91]:
current_date

Timestamp('2025-03-04 07:28:40.839943+0000', tz='Etc/UTC')

In [92]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-04 06:00:00
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-04 06:00:00
2,2,1,11,7,3,2,3,2,1,7,...,1,1,4,4,3,0,0,1,4,2025-03-04 06:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-04 06:00:00
4,2,2,1,2,3,2,3,1,2,4,...,1,0,0,1,0,0,0,0,7,2025-03-04 06:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,259,2025-03-04 06:00:00
247,1,4,2,0,1,1,1,0,1,1,...,0,1,0,1,0,0,0,1,260,2025-03-04 06:00:00
248,0,4,5,16,21,34,25,31,37,28,...,8,10,1,0,0,1,1,1,261,2025-03-04 06:00:00
249,63,126,177,156,102,81,88,81,79,97,...,14,9,2,2,1,4,4,16,262,2025-03-04 06:00:00


In [93]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-04 02:29:00,828 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 02:29:00,832 INFO: Initializing external client
2025-03-04 02:29:00,832 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-04 02:29:01,480 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214668


In [94]:
print(features.columns)

Index(['rides_t-672', 'rides_t-671', 'rides_t-670', 'rides_t-669',
       'rides_t-668', 'rides_t-667', 'rides_t-666', 'rides_t-665',
       'rides_t-664', 'rides_t-663',
       ...
       'rides_t-8', 'rides_t-7', 'rides_t-6', 'rides_t-5', 'rides_t-4',
       'rides_t-3', 'rides_t-2', 'rides_t-1', 'pickup_location_id',
       'pickup_hour'],
      dtype='object', length=674)


In [95]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [79]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,1.0
3,6,0.0
4,7,2.0
...,...,...
246,259,1.0
247,260,1.0
248,261,1.0
249,262,68.0


In [80]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 186, 100, 162, 262,  48, 239, 236, 237, 238], dtype=int32)